In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [46]:
#hyper parameters

input_size = 784
num_classes = 10
batch_size = 100
num_epochs = 20
learning_rate= 0.001

In [47]:
#MNIST

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [49]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()   
        self.conv1 = nn.Conv2d(1, 32, 5)     
        self.pool = nn.MaxPool2d(2,2)       
        self.conv2 = nn.Conv2d(32, 64, 5)    
        self.fc1 = nn.Linear(64 * 4 * 4, 204) 
        self.fc2 = nn.Linear(204, 40)
        self.fc3 = nn.Linear(40, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = NeuralNet()
print(model)
    
        

NeuralNet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=204, bias=True)
  (fc2): Linear(in_features=204, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=10, bias=True)
)


In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [53]:
#training
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        output = model(images)
        loss = criterion(output, labels)
        
        #backward and optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
        
    
    
    
    

Epoch [1/20], Step [100/600], Loss: 0.2825
Epoch [1/20], Step [200/600], Loss: 0.2077
Epoch [1/20], Step [300/600], Loss: 0.4602
Epoch [1/20], Step [400/600], Loss: 0.2970
Epoch [1/20], Step [500/600], Loss: 0.4226
Epoch [1/20], Step [600/600], Loss: 0.3333
Epoch [2/20], Step [100/600], Loss: 0.3200
Epoch [2/20], Step [200/600], Loss: 0.4940
Epoch [2/20], Step [300/600], Loss: 0.2785
Epoch [2/20], Step [400/600], Loss: 0.4899
Epoch [2/20], Step [500/600], Loss: 0.3926
Epoch [2/20], Step [600/600], Loss: 0.3056
Epoch [3/20], Step [100/600], Loss: 0.5266
Epoch [3/20], Step [200/600], Loss: 0.2517
Epoch [3/20], Step [300/600], Loss: 0.3322
Epoch [3/20], Step [400/600], Loss: 0.2052
Epoch [3/20], Step [500/600], Loss: 0.3515
Epoch [3/20], Step [600/600], Loss: 0.4042
Epoch [4/20], Step [100/600], Loss: 0.1935
Epoch [4/20], Step [200/600], Loss: 0.3474
Epoch [4/20], Step [300/600], Loss: 0.2323
Epoch [4/20], Step [400/600], Loss: 0.3373
Epoch [4/20], Step [500/600], Loss: 0.3849
Epoch [4/20

In [54]:
#testing

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    

Accuracy of the network: 95.39 %
